In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Initial condition is some distribution $f_0(r, \vec{v}) = f(r, \vec{v}, 0)$ (which can equivalently be expressed in terms of energy and angular momentum)

### Plummer model
### $\rho_P(r) = \frac{3M_0}{4\pi r_0^3}\left(1 + \frac{r^2}{r_0^2}\right)^{-5/2}$
### $\Phi_P(r) = \frac{GM_0}{\sqrt{r^2 + r_0^2}}$
### Black hole nuclear cluster
### $\Phi_{NC}(r) = \frac{GM_{\rm BH}}{r}$

### Energy / Angular Momentum Bounds
### $E = \Phi(r) - \frac{1}{2}v^2$
### $E_{\rm min} = 0$
### $E_{\rm max} = \Phi_P(r=0) = \frac{-GM_0}{r_0}$
### $R \equiv J^2/J_c^2$
### $J_c$ is the specific angular momentum of a circular orbit, so even though $J = J(E)$, $R$ is always bounded between $0$ and $1$
### $J = r|\hat{r}\times\vec{v}|$
### $R_{\rm min} = 0$
### $R_{\rm max} = 1$

### How to convert between code grid $E$, $R$ and physical position $r$, velocity vector $\vec{v}$?
### $N(E,R)dEdR = \int_{r_-}^{r_+} 4\pi r^2 dr(fd^3v)$ (Eq. 17 of Cohn+Kulsrud 1978)

In [ ]:
def plummer_distribution(r, r0, M0):
    # https://en.wikipedia.org/wiki/Plummer_model#:~:text=The%20Plummer%20model%20or%20Plummer,body%20simulations%20of%20stellar%20systems.
    return (3*M0/(4*np.pi*r0**3.))*(1. + r*r/r0/r0)**(-5./2.)

def bh_potential(r, Mbh):
    return Mbh/r



In [4]:
def generate_distribution(rmin, rmax, r0, p_type='plummer', v_type='isotropic'):
    
    if p_type == 'plummer':
        rho = plummer_distribution(r, r0, 1)

SyntaxError: unexpected EOF while parsing (<ipython-input-4-8715dccb060c>, line 1)

### Equations 3-4 of of Takahashi 1995
### $A\frac{\partial f}{\partial t} = - \frac{\partial F_E}{\partial E} - \frac{\partial F_R}{\partial R}$
### $-F_E = D_{EE}\frac{\partial f}{\partial E} + D_{ER}\frac{\partial f}{\partial R} + D_E f$
### $-F_R = D_{RE}\frac{\partial f}{\partial E} + D_{RR}\frac{\partial f}{\partial R} + D_R f$
### Appendix C of Cohn 1979
### $D_{EE} = \frac{8\pi^2}{3}J_c^2(E) \int \frac{dr}{v_r}v^2(F_0 + F_2)$
### $D_E = -8\pi^2J_c^2(E) \int \frac{dr}{v_r}F_1$
### $D_{ER} = D_{RE} = \frac{16\pi^2}{3}J^2\int \frac{dr}{v_r} \left(\frac{v^2}{v_c^2} - 1\right)(F_0 + F_2)$
### $D_{RR} = \frac{16\pi^2}{3}R\int\frac{dr}{v^2}\left\{2\frac{r^2}{v^2}\left[v_t^2\left(\frac{v^2}{v_c^2} - 1\right)^2 + v_r^2\right]F_0 + 3\frac{r^2v_r^2}{v^2}F_1 + \frac{r^2}{v^2}\left[2v_t^2\left(\frac{v^2}{v_c^2}-1\right)^2 - v_r^2\right]F_2\right\}$
### $D_R    = -16\pi^2 R r_c^2 \int \frac{dr}{v_r}\left(1 - \frac{v_c^2}{v^2}\right)F_1$
### Appendix B of Cohn 1979
### $F_0(E,r) = 4\pi\Gamma \int_{-\infty}^E dE' \bar{f}(E',r)$
### $F_1(E,r) = 4\pi\Gamma \int_E^\phi dE'\bar{f}(E',r)\left(\frac{\phi - E'}{\phi - E}\right)^{1/2}$
### $F_2(E,r) = 4\pi\Gamma \int_E^\phi dE'\bar{f}(E',r)\left(\frac{\phi-E'}{\phi - E}\right)^{3/2}$
### $\Gamma = 4\pi G^2m^2 {\rm ln}\Lambda$
### ${\rm ln}\Lambda$ ~ logarithmic cutoff factor (?)
### Equation 16 of Cohn 1979
### $\bar{f}(E,r) = \frac{1}{2R_{\rm max}^{1/2}}\int_0^{R_{\rm max}} \frac{dR}{(R_{\rm max} - R)^{1/2}}f(E,R)$
### $R_{\rm max}(E,r) = 2r^2[\Phi(r) - E]/J_c^2$

### We need to choose a radial grid to construct $\bar{f}(E,r)$. This will later be used if we solve the Poisson equation, as well. 
### For a Plummer potential we can follow Takahashi 1995, who take $r_{\rm min} = 10^{-8}\,r_0$ and $r_{\rm max} = 100\,r_0$, where $r_0$ is the core radius of the Plummer potential
### For a central SMBH, we can follow Cohn 1978, who ()

In [ ]:
from scipy.integrate import trapz

def calc_fbar(f,grid_r):
    
    grid_E = f.domain.grids[0]
    grid_R = f.domain.grids[1]
    fbar_domain = field.Domain((grid_E, grid_r))
    fbar = field.Field(fbar_domain)
    
    E_data = grid_E.values[:]
    R_data = grid_R.values[:]
    r_data = grid_r.values[:]
    
    
    # Construct an integrand that is a 3D grid in (E, r, R) space; trapz() will integrate out the R dimension. 
    integrand_r = r_data[None,:,None]
    integrand_E = E_data[:,None,None]
    
    Jc   = np.sqrt(dpotential_dr(integrand_r)*integrand_r**3.)
    Rmax = 2*integrand_r*(potential(integrand_r)-integrand_E)/Jc**2.
    
    # R varies from 0 to Rmax. Assume R grid starts at 0!
    ind_Rmax = np.argmin(np.abs(R_data - Rmax))
    integrand_R = R_data[None,None,:ind_Rmax]
    integrand_f = f.values[:,None,:ind_Rmax] # add middle axis to f for r dimension
    integrand = integrand_f/(Rmax - integrand_R)**1.5
    
    # for output data
    np.copyto(fbar.data, (1./2./Rmax[:,:,0]**0.5)*trapz(integrand, integrand_R, axis=-1))
    
def calc_F012(fbar):
    Gamma = 4*pi*G*G*m*m*np.log(Lambda) # Lambda ~ logarithmic cutoff factor, G ~ gravitational constant, m ~ mass of individual star
    
    
    grid_E = fbar.domain.grids[0]
    grid_r = fbar.domain.grids[1]
    fbar_domain = field.Domain((grid_E, grid_r))
    fbar = field.Field(fbar_domain)
 
    # Check to make sure arrays dont occupy same place in memory. the '[:]' part does this for us, I think?
    E_data = grid_E.values[:]
    r_data = grid_r.values[:]  
    Eprime_data = grid_E.values[:]
    

In [56]:
x = np.arange(0,10)
y = np.arange(0,10)

print(x,y)
X, Y = np.meshgrid(x,y)

R = 1./(1. + Y**2.)


from scipy.integrate import trapz

#print(R.shape)
#print(X.shape)

#print(trapz(np.ones(np.shape(x)),x))

I = trapz(np.ones(np.shape(X)), X,axis=-1)

print(X)
print(I)
print(I.shape)

[0 1 2 3 4 5 6 7 8 9] [0 1 2 3 4 5 6 7 8 9]
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
[9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]
(10,)


In [60]:
X_new = X[:,:,None]
print(X_new.shape,X_new[:,:,0].shape)

(10, 10, 1) (10, 10)
